# パスワード設定

In [1]:
from IPython.lib import passwd
passwd()

quit()

Enter password:  ·········
Verify password:  ·········


# Athenaクエリサンプル

In [ ]:
from pyathena import connect

cursor = connect(s3_staging_dir="s3://aws-athena-query-results-{account_id}-ap-northeast-1/jupyter/",
                 region_name="ap-northeast-1").cursor()
cursor.execute('SELECT * FROM "athena_cur_billing"."billing" limit 10;')
print(cursor.description)
print(cursor.fetchall())


# AWS費用の月次比較：2021/10~2021/11

In [ ]:
import pandas as pd
from pyathena import connect

conn = connect(s3_staging_dir="s3://aws-athena-query-results-{account_id}-ap-northeast-1/jupyter/",
                 region_name="ap-northeast-1")
sql = '''
SELECT line_item_product_code, 
    ROUND(SUM(CASE WHEN year = '2021' AND month = '11' AND line_item_blended_cost > 0 THEN
    line_item_blended_cost
    ELSE 0 END), 1) as this_month,
    ROUND(SUM(CASE WHEN year = '2021' AND month = '10' AND line_item_blended_cost > 0 THEN
    line_item_blended_cost
    ELSE 0 END), 1) as last_month,
    (ROUND(SUM(CASE WHEN year = '2021' AND month = '11' AND line_item_blended_cost > 0 THEN
    line_item_blended_cost
    ELSE 0 END), 1) / 
    ROUND(SUM(CASE WHEN year = '2021' AND month = '10' AND line_item_blended_cost > 0 THEN
    line_item_blended_cost
    ELSE 0 END), 1)
    * 100) as ratio
    from "athena_cur_billing"."billing"
    WHERE line_item_blended_cost > 0
    GROUP BY line_item_product_code
    ORDER BY ratio ASC
'''

df = pd.read_sql(sql, conn)
display(df)